In [13]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

In [14]:
relevant_columns = [
    "main.disorder",  # Target variable

    # Schizophrenia (alpha PSD)
    "AB.C.alpha.a.FP1", "AB.C.alpha.b.FP2", "AB.C.alpha.c.F7", "AB.C.alpha.d.F3",
    "AB.C.alpha.e.Fz", "AB.C.alpha.f.F4", "AB.C.alpha.g.F8", "AB.C.alpha.h.T3",
    "AB.C.alpha.i.C3", "AB.C.alpha.j.Cz", "AB.C.alpha.k.C4", "AB.C.alpha.l.T4",
    "AB.C.alpha.m.T5", "AB.C.alpha.n.P3", "AB.C.alpha.o.Pz", "AB.C.alpha.p.P4",
    "AB.C.alpha.q.T6", "AB.C.alpha.r.O1", "AB.C.alpha.s.O2",

    # Trauma and stress-related disorders (beta FC)
    "COH.D.beta.a.FP1.b.FP2", "COH.D.beta.c.F7.d.F3", "COH.D.beta.e.Fz.f.F4",
    "COH.D.beta.g.F8.h.T3", "COH.D.beta.i.C3.j.Cz", "COH.D.beta.k.C4.l.T4",
    "COH.D.beta.m.T5.n.P3", "COH.D.beta.o.Pz.p.P4", "COH.D.beta.q.T6.r.O1",

    # Anxiety disorders (whole band PSD: all frequency bands)
    "AB.A.delta.a.FP1", "AB.A.delta.b.FP2", "AB.B.theta.a.FP1", "AB.B.theta.b.FP2",
    "AB.C.alpha.a.FP1", "AB.C.alpha.b.FP2", "AB.D.beta.a.FP1", "AB.D.beta.b.FP2",
    "AB.E.highbeta.a.FP1", "AB.E.highbeta.b.FP2", "AB.F.gamma.a.FP1", "AB.F.gamma.b.FP2",

    # Mood disorders (theta FC)
    "COH.B.theta.a.FP1.b.FP2", "COH.B.theta.c.F7.d.F3", "COH.B.theta.e.Fz.f.F4",
    "COH.B.theta.g.F8.h.T3", "COH.B.theta.i.C3.j.Cz", "COH.B.theta.k.C4.l.T4",
    "COH.B.theta.m.T5.n.P3", "COH.B.theta.o.Pz.p.P4", "COH.B.theta.q.T6.r.O1",

    # Addictive disorders (theta PSD)
    "AB.B.theta.a.FP1", "AB.B.theta.b.FP2", "AB.B.theta.c.F7", "AB.B.theta.d.F3",
    "AB.B.theta.e.Fz", "AB.B.theta.f.F4", "AB.B.theta.g.F8", "AB.B.theta.h.T3",
    "AB.B.theta.i.C3", "AB.B.theta.j.Cz", "AB.B.theta.k.C4", "AB.B.theta.l.T4",
    "AB.B.theta.m.T5", "AB.B.theta.n.P3", "AB.B.theta.o.Pz", "AB.B.theta.p.P4",
    "AB.B.theta.q.T6", "AB.B.theta.r.O1", "AB.B.theta.s.O2",

    # Obsessive-compulsive disorder (gamma FC)
    "COH.F.gamma.a.FP1.b.FP2", "COH.F.gamma.c.F7.d.F3", "COH.F.gamma.e.Fz.f.F4",
    "COH.F.gamma.g.F8.h.T3", "COH.F.gamma.i.C3.j.Cz", "COH.F.gamma.k.C4.l.T4",
    "COH.F.gamma.m.T5.n.P3", "COH.F.gamma.o.Pz.p.P4", "COH.F.gamma.q.T6.r.O1"
]


In [15]:
# Load dataset with only relevant columns
data = pd.read_csv("data/train.csv", usecols=relevant_columns)

# Filter only Mood Disorder and Healthy Control
data = data[data["main.disorder"].isin(["Healthy control", "Obsessive compulsive disorder"])]

# Encode labels manually: Healthy Control -> 0, Mood Disorder -> 1
data["main.disorder"] = data["main.disorder"].map({
    "Healthy control": 0,
    "Obsessive compulsive disorder": 1
})

# Features and labels
X = data.drop(columns=["main.disorder"]).values
y = data["main.disorder"].values.astype(int)  # Ensure integer labels

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save the scaler
joblib.dump(scaler, "svm_ocd_scaler.pkl")

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [16]:
# ---------------- Hyperparameter Tuning (Grid Search) ----------------
param_grid = {
    "C": [0.1, 0.5, 1, 5, 10],  # Regularization parameter
    "kernel": ["linear", "rbf"],  # Linear and RBF kernel
}

svm = SVC(probability=True, class_weight="balanced", random_state=42)

grid_search = GridSearchCV(svm, param_grid, cv=5, scoring="roc_auc", n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Get the best model
best_svm = grid_search.best_estimator_

# Save the trained SVM model
joblib.dump(best_svm, "svm_ocd_tuned.pkl")

Fitting 5 folds for each of 10 candidates, totalling 50 fits


['svm_ocd_tuned.pkl']

In [17]:
# ---------------- Evaluate Best Model ----------------
y_pred = best_svm.predict(X_test)
y_pred_prob = best_svm.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_prob)

print(f"\n Tuned SVM Model for Mood Disorder vs. Healthy Control")
print(f" Best Parameters: {grid_search.best_params_}")
print(f" Accuracy: {accuracy:.4f}")
print(f" AUC Score: {auc:.4f}")


 Tuned SVM Model for Mood Disorder vs. Healthy Control
 Best Parameters: {'C': 0.5, 'kernel': 'linear'}
 Accuracy: 0.4400
 AUC Score: 0.4123


In [18]:
def predict_mood_disorder(new_data_path):
    """
    Loads new EEG data, scales it, and predicts Mood Disorder probability.
    """
    # Load new data
    new_data = pd.read_csv(new_data_path, usecols=relevant_columns[1:])  # Exclude "main.disorder"
    
    # Load saved scaler and model
    scaler = joblib.load("svm_ocd_scaler.pkl")
    svm = joblib.load("svm_ocd_tuned.pkl")

    # Load new data
    new_data = pd.read_csv(new_data_path, usecols=relevant_columns[1:])  # Exclude "main.disorder"

# Convert DataFrame to NumPy before applying StandardScaler
    new_data_scaled = scaler.transform(new_data.values)  # <-- FIXED


    # Predict probabilities
    predictions = svm.predict_proba(new_data_scaled)[:, 1]  # Probability of Mood Disorder

    return predictions

In [19]:
preds = predict_mood_disorder("data.csv")


In [20]:
from sklearn.metrics import roc_curve

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Find the optimal threshold (Youden’s J statistic: max(TPR - FPR))
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

print(f"🔹 Optimal Decision Threshold: {optimal_threshold:.4f}")

# Convert probabilities to labels using the optimal threshold
pred_labels = (y_pred_prob >= optimal_threshold).astype(int)


🔹 Optimal Decision Threshold: 0.2901


In [21]:
print(preds)

[0.34915523 0.29386753 0.31366611 0.33739844 0.29391479 0.32910641
 0.31430375 0.31759785 0.30809422 0.32316028 0.34848222 0.31583583
 0.31896696 0.35696479 0.28684658 0.34675416 0.3179949  0.34435402
 0.33117706 0.30520986 0.32206545 0.29891281 0.3370303  0.28419234
 0.33373973 0.34211548 0.31188116 0.3192694  0.30545728 0.31687178
 0.30995607 0.31195469 0.31188651 0.33572871 0.39368266 0.31281155
 0.33214286 0.35232142 0.30052299 0.28185021 0.3407167  0.33331952
 0.31584724 0.27343537 0.34662673 0.31000143 0.28368041 0.30472816
 0.29220048 0.31636769 0.31528516 0.29591123 0.29186943 0.34243257
 0.30555373 0.30901137 0.40742475 0.32312757 0.30985525 0.29750107
 0.30396554 0.31690787 0.32200747 0.36043633 0.33305634 0.31219759
 0.34890325 0.29731857 0.31807179 0.31878576 0.3433311  0.32162411
 0.3274928  0.36410841 0.30538031 0.31487372 0.32291618 0.293354
 0.35406717 0.29046998 0.31786004 0.35320929 0.28764534 0.33122661
 0.29419075 0.32256864 0.3363495  0.30404823 0.31140398 0.343251

In [22]:
def decode_predictions(preds):
    """
    Convert predictions (0,1) back to disorder labels.
    """
    label_map = {0: "Healthy control", 1: "Obsessive compulsive disorder"}
    return [label_map[pred] for pred in preds]

# Example usage
pred_labels = (preds >= 0.2901).astype(int)  # Convert probabilities to binary labels
decoded_labels = decode_predictions(pred_labels)

print("\nPredicted Labels:\n", decoded_labels)


Predicted Labels:
 ['Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Healthy control', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Healthy control', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compulsive disorder', 'Obsessive compul

In [23]:
import pandas as pd

# Convert predictions into a DataFrame
output_df = pd.DataFrame({"Predicted Label": decoded_labels})

# Save to CSV
output_df.to_csv("predicted_ocd_labels.csv", index=False)

print(" Predictions saved to 'predicted_labels.csv'")


 Predictions saved to 'predicted_labels.csv'
